In [129]:
# Dependencies
# ----------------------------------
from sqlalchemy import create_engine, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base

from sqlalchemy import Column, Integer, String, Float, DateTime, Date, Time, func

In [130]:
import datetime as dt

In [133]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [134]:
# Creates Classes which will serve as the anchor points for our Table
class Update(Base):
    __tablename__ = 'database_update'
    id = Column(Integer, primary_key=True)
    update_time = Column(Time)
    update_date = Column(Date)
    update_datetime = Column(DateTime)
    update_type = Column(String(255))
    candidate_id_str = Column(String(255))

In [135]:
# Create Database Connection
engine = create_engine("sqlite:///../db/twitter_db.sqlite")
conn = engine.connect()

In [136]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [137]:
# help(Base.metadata)

In [143]:
# Base.metadata.reflect(engine)

In [139]:
# Base.metadata.clear()

In [142]:
for table in Base.metadata.tables:
    print(table)

database_update
tweet_data


In [124]:
Base.metadata.tables

immutabledict({'database_update': Table('database_update', MetaData(bind=None), Column('id', Integer(), table=<database_update>, primary_key=True, nullable=False), Column('update_time', Time(), table=<database_update>), Column('update_date', Date(), table=<database_update>), Column('update_datetime', DateTime(), table=<database_update>), Column('update_type', String(length=255), table=<database_update>), Column('candidate_name', String(length=255), table=<database_update>), Column('candidate_id_str', String(length=255), table=<database_update>), schema=None)})

In [144]:
# Base.metadata.remove(table = Update())

In [145]:
# help(Table)

In [146]:
# Base_AutoMap.metadata.remove(table = Update)

In [ ]:
# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [125]:
Base.metadata.drop_all(engine)

In [147]:
from sqlalchemy import inspect

In [148]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [149]:
# Collect the names of tables within the database
inspector.get_table_names()

['database_update', 'tweet_data']

In [151]:
# Using the inspector to print the column names within the 'tweet_data' table and its types
columns = inspector.get_columns('database_update')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
update_time TIME
update_date DATE
update_datetime DATETIME
update_type VARCHAR(255)
candidate_id_str VARCHAR(255)


## Testing

In [35]:
Base.prepare(engine, reflect=True)

AttributeError: type object 'Base' has no attribute 'prepare'

In [29]:
Tweets = Base.classes.tweet_data


AttributeError: type object 'Base' has no attribute 'classes'

In [24]:
session.query(Tweets.tweet_id).limit(50).all()

NameError: name 'Tweets' is not defined

In [ ]:
date_1 = dt.date(2019, 7, 30)
date_2 = dt.date(2019, 7, 15)
date_3 = dt.date(2019, 6, 30)

In [22]:
from sqlalchemy.orm import Session

In [23]:
session = Session(engine)

In [ ]:
session.add(Update_Record(updated_date = date_1))

In [ ]:
session.new

In [ ]:
session.commit()

In [ ]:
session.query(Update_Record.updated_date).all()

In [ ]:
session.add(Update_Record(updated_date = date_2))

In [ ]:
session.add(Update_Record(updated_date = date_3))

In [ ]:
session.new

In [ ]:
session.commit()

In [ ]:
session.query(Update_Record.updated_date).all()

In [ ]:
session.query(func.max(Update_Record.updated_date)).first()

In [ ]:
date_4 = dt.date(2019, 8, 1)

In [ ]:
session.add(Update_Record(updated_date = date_4))

In [ ]:
session.commit()

In [ ]:
session.query(func.max(Update_Record.updated_date)).first()

In [ ]:
session.query(Update_Record.updated_date).all()